# Projeto de Análise de Dados
# Desenvolvido por Ian Rodrigues dos Reis Paixão
# Contato: iang.master100@gmail.com

O objetivo deste projeto é validar a integridade do Balancete de um grupo econômico, considerando as movimentações contábeis organizadas por ativo e passivo.

As etapas do problema incluem:

*   1️⃣ Limpeza e preparação de dados: Importar dados de diversas planilhas CSV, tratar inconsistências, atualizar valores nulos e remover duplicados.

*   2️⃣ Análise temporal: Verificar a evolução do Balancete, observando o comportamento do ativo e do passivo.
*   3️⃣ Análise por categoria: Consolidar o Balancete em categorias de ativo e passivo, garantindo a integridade do grupo econômico.


*   4️⃣ Exportação de resultados: Gerar um relatório em formato JSON com a análise do Balancete.

*  5️⃣ Visualização: Exibir os resultados de maneira interativa utilizando a biblioteca 'json2html'.


✨ Resultado: Um fluxo automatizado e confiável, ideal para auditorias financeiras, oferecendo uma solução eficiente e elegante


In [ ]:
'''
-- Exercício 1: Modelagem e Tratamento de Dados

-- A) Criação do Diagrama de Entidade e Relacionamento (DER)

CREATE TABLE Empresa (
    Codigo INT PRIMARY KEY,
    Empresa VARCHAR(255) NOT NULL
);

CREATE TABLE MovimentoContabil (
    Data DATE NOT NULL,
    `Conta Contábil` VARCHAR(50) NOT NULL,
    Descricao VARCHAR(255) NOT NULL,
    Valor DECIMAL(15,2) NOT NULL,
    Relatorio INT NOT NULL,
    `Centro de Custo` INT NOT NULL,
    Empresa INT NOT NULL,
    Seq INT PRIMARY KEY
);

CREATE TABLE Relatorio (
    Código INT PRIMARY KEY,
    Relatorio VARCHAR(255) NOT NULL,
    `Indicador Ativo / Inativo` VARCHAR(10) NOT NULL
);

CREATE TABLE CentroCusto (
    Código INT PRIMARY KEY,
    Descrição VARCHAR(255) NOT NULL,
    `Indicador Ativo / Inativo` VARCHAR(10) NOT NULL
);

-- B) Tratamento dos dados

-- Atualizar valores nulos para zero nas tabelas necessárias
UPDATE MovimentoContabil
SET Valor = 0
WHERE Valor IS NULL;

-- Remover registros duplicados na MovimentoContabil
DELETE mc1
FROM MovimentoContabil mc1
INNER JOIN MovimentoContabil mc2
    ON mc1.Seq > mc2.Seq
    AND mc1.`Conta Contábil` = mc2.`Conta Contábil`
    AND mc1.Empresa = mc2.Empresa;

-- C) Scripts para popular as tabelas conforme o arquivo Excel

-- Cada Planilha do excel foi Salva como um arquivo csv separadamente

-- Carregar dados na tabela Empresa
LOAD DATA INFILE 'C:/Users/iangm/Downloads/Empresa.csv'
INTO TABLE Empresa
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;

-- Carregar dados na tabela MovimentoContabil
LOAD DATA INFILE 'C:/Users/iangm/Downloads/Movimento Contábil.csv'
INTO TABLE MovimentoContabil
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;

-- Carregar dados na tabela Relatorio
LOAD DATA INFILE 'C:/Users/iangm/Downloads/Relatório.csv'
INTO TABLE Relatorio
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;

-- Carregar dados na tabela CentroCusto
LOAD DATA INFILE 'C:/Users/iangm/Downloads/Centro de Custo.csv'
INTO TABLE CentroCusto
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS;

-- Exercício 2: Análise e Exportação para JSON

-- A) Validação da integridade do Balancete considerando Ativo e Passivo do grupo econômico

SELECT
    SUM(CASE WHEN LEFT(`Conta Contábil`, 1) = '1' THEN Valor ELSE 0 END) AS total_ativo,
    SUM(CASE WHEN LEFT(`Conta Contábil`, 1) != '1' THEN Valor ELSE 0 END) AS total_passivo,
    (SUM(CASE WHEN LEFT(`Conta Contábil`, 1) = '1' THEN Valor ELSE 0 END) +
     SUM(CASE WHEN LEFT(`Conta Contábil`, 1) != '1' THEN Valor ELSE 0 END)) AS integridade
FROM MovimentoContabil;

-- Regra: Integridade é quando TOTAL ATIVO + TOTAL PASSIVO = 0

-- B) Consulta para retornar JSON com a estrutura solicitada para o grupo econômico

SELECT
    JSON_OBJECT(
        'total_ativo', SUM(CASE WHEN LEFT(`Conta Contábil`, 1) = '1' THEN Valor ELSE 0 END),
        'total_passivo', SUM(CASE WHEN LEFT(`Conta Contábil`, 1) != '1' THEN Valor ELSE 0 END),
        'integridade', (SUM(CASE WHEN LEFT(`Conta Contábil`, 1) = '1' THEN Valor ELSE 0 END) +
                        SUM(CASE WHEN LEFT(`Conta Contábil`, 1) != '1' THEN Valor ELSE 0 END))
    ) AS resultado_json
FROM MovimentoContabil;

'''

"\n-- Exercício 1: Modelagem e Tratamento de Dados\n\n-- A) Criação do Diagrama de Entidade e Relacionamento (DER)\n\nCREATE TABLE Empresa (\n    Codigo INT PRIMARY KEY,\n    Empresa VARCHAR(255) NOT NULL\n);\n\nCREATE TABLE MovimentoContabil (\n    Data DATE NOT NULL,\n    `Conta Contábil` VARCHAR(50) NOT NULL,\n    Descricao VARCHAR(255) NOT NULL,\n    Valor DECIMAL(15,2) NOT NULL,\n    Relatorio INT NOT NULL,\n    `Centro de Custo` INT NOT NULL,\n    Empresa INT NOT NULL,\n    Seq INT PRIMARY KEY\n);\n\nCREATE TABLE Relatorio (\n    Código INT PRIMARY KEY,\n    Relatorio VARCHAR(255) NOT NULL,\n    `Indicador Ativo / Inativo` VARCHAR(10) NOT NULL\n);\n\nCREATE TABLE CentroCusto (\n    Código INT PRIMARY KEY,\n    Descrição VARCHAR(255) NOT NULL,\n    `Indicador Ativo / Inativo` VARCHAR(10) NOT NULL\n);\n\n-- B) Tratamento dos dados\n\n-- Atualizar valores nulos para zero nas tabelas necessárias\nUPDATE MovimentoContabil\nSET Valor = 0\nWHERE Valor IS NULL;\n\n-- Remover registros du

In [ ]:
import pandas as pd
import sqlite3
import json
import IPython.display as display
from json2html import *

# Caminhos para os arquivos CSV no Google Colab
empresa_csv = '/content/Empresa.csv'
movimento_contabil_csv = '/content/Movimento Contábil.csv'
relatorio_csv = '/content/Relatório.csv'
centro_custo_csv = '/content/Centro de Custo.csv'

# Criando o banco de dados SQLite
conn = sqlite3.connect('/content/balancete.db')
cursor = conn.cursor()

# Criando as tabelas SQLite
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Empresa (
        Codigo INT PRIMARY KEY,
        Empresa TEXT NOT NULL
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS MovimentoContabil (
        Data TEXT NOT NULL,
        "Conta Contábil" TEXT NOT NULL,
        Descricao TEXT NOT NULL,
        Valor REAL NOT NULL,
        Relatorio INT NOT NULL,
        "Centro de Custo" INT NOT NULL,
        Empresa INT NOT NULL,
        Seq INT PRIMARY KEY
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Relatorio (
        Código INT PRIMARY KEY,
        Relatorio TEXT NOT NULL,
        "Indicador Ativo / Inativo" TEXT NOT NULL
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CentroCusto (
        Código INT PRIMARY KEY,
        Descrição TEXT NOT NULL,
        "Indicador Ativo / Inativo" TEXT NOT NULL
    );
''')

print("Tabelas criadas com sucesso!")

# Carregar os arquivos CSV para DataFrames
df_empresa = pd.read_csv(empresa_csv)
df_movimento_contabil = pd.read_csv(movimento_contabil_csv)
df_relatorio = pd.read_csv(relatorio_csv)
df_centro_custo = pd.read_csv(centro_custo_csv)

# Enviando os DataFrames para o banco de dados SQLite
df_empresa.to_sql('Empresa', conn, if_exists='replace', index=False)
df_movimento_contabil.to_sql('MovimentoContabil', conn, if_exists='replace', index=False)
df_relatorio.to_sql('Relatorio', conn, if_exists='replace', index=False)
df_centro_custo.to_sql('CentroCusto', conn, if_exists='replace', index=False)

print("Dados inseridos com sucesso!")

# Tratamento dos dados via SQL
cursor.execute('''
    UPDATE MovimentoContabil
    SET Valor = 0
    WHERE Valor IS NULL;
''')

cursor.execute('''
    DELETE FROM MovimentoContabil
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM MovimentoContabil
        GROUP BY "Conta Contábil", Empresa, Seq
    );
''')

conn.commit()
print("Tratamento dos dados realizado com sucesso!")

# Consulta SQL para gerar o JSON estruturado
query_json = """
SELECT
    e.Empresa AS empresa,
    SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) = '1' THEN mc.Valor ELSE 0 END) AS total_ativo,
    SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) != '1' THEN mc.Valor ELSE 0 END) AS total_passivo,
    (SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) = '1' THEN mc.Valor ELSE 0 END) +
     SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) != '1' THEN mc.Valor ELSE 0 END)) AS check_integridade
FROM MovimentoContabil mc
JOIN Empresa e ON mc.Empresa = e.Codigo
GROUP BY e.Empresa;
"""

df_json = pd.read_sql_query(query_json, conn)

# Converter para JSON
result_json = df_json.to_json(orient='records', indent=4)

# Salvar JSON
json_path = '/content/resultado_balancete.json'
with open(json_path, 'w') as f:
    f.write(result_json)

print(f"Arquivo JSON gerado com sucesso! Caminho: {json_path}")

# Abrir o JSON em uma visualização HTML interativa
with open(json_path, 'r') as f:
    json_content = json.load(f)
    html_content = json2html.convert(json = json_content)
    display.display(display.HTML(html_content))

# Fechar a conexão com o banco de dados
conn.close()


Tabelas criadas com sucesso!
Dados inseridos com sucesso!
Tratamento dos dados realizado com sucesso!
Arquivo JSON gerado com sucesso! Caminho: /content/resultado_balancete.json


empresa,total_ativo,total_passivo,check_integridade
EMPRESA 1,2025109140.4,12237427.98,2037346568.38
EMPRESA 2,184290832.0399999,56767969.66000001,241058801.69999993
EMPRESA 3,-7051.2,-9924192.03,-9931243.229999999
EMPRESA 4,1497706894.3999999,1160661.7199999997,1498867556.12
EMPRESA 5,1876694707.36,-323909775.46,1552784931.8999999


In [ ]:
# Autor: Ian Rodrigues

import pandas as pd
import sqlite3
import json
import IPython.display as display
from json2html import *

# Caminhos para os arquivos CSV no Google Colab
empresa_csv = '/content/Empresa.csv'
movimento_contabil_csv = '/content/Movimento Contábil.csv'
relatorio_csv = '/content/Relatório.csv'
centro_custo_csv = '/content/Centro de Custo.csv'

# Criando o banco de dados SQLite
conn = sqlite3.connect('/content/balancete.db')
cursor = conn.cursor()

# Criando as tabelas SQLite
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Empresa (
        Codigo INT PRIMARY KEY,
        Empresa TEXT NOT NULL
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS MovimentoContabil (
        Data TEXT NOT NULL,
        "Conta Contábil" TEXT NOT NULL,
        Descricao TEXT NOT NULL,
        Valor REAL NOT NULL,
        Relatorio INT NOT NULL,
        "Centro de Custo" INT NOT NULL,
        Empresa INT NOT NULL,
        Seq INT PRIMARY KEY
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Relatorio (
        Código INT PRIMARY KEY,
        Relatorio TEXT NOT NULL,
        "Indicador Ativo / Inativo" TEXT NOT NULL
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CentroCusto (
        Código INT PRIMARY KEY,
        Descrição TEXT NOT NULL,
        "Indicador Ativo / Inativo" TEXT NOT NULL
    );
''')

print("Tabelas criadas com sucesso!")

# Carregar os arquivos CSV para DataFrames
df_empresa = pd.read_csv(empresa_csv)
df_movimento_contabil = pd.read_csv(movimento_contabil_csv)
df_relatorio = pd.read_csv(relatorio_csv)
df_centro_custo = pd.read_csv(centro_custo_csv)

# Enviando os DataFrames para o banco de dados SQLite
df_empresa.to_sql('Empresa', conn, if_exists='replace', index=False)
df_movimento_contabil.to_sql('MovimentoContabil', conn, if_exists='replace', index=False)
df_relatorio.to_sql('Relatorio', conn, if_exists='replace', index=False)
df_centro_custo.to_sql('CentroCusto', conn, if_exists='replace', index=False)

print("Dados inseridos com sucesso!")

# Tratamento dos dados via SQL
cursor.execute('''
    UPDATE MovimentoContabil
    SET Valor = 0
    WHERE Valor IS NULL;
''')

cursor.execute('''
    DELETE FROM MovimentoContabil
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM MovimentoContabil
        GROUP BY "Conta Contábil", Empresa, Seq
    );
''')

conn.commit()
print("Tratamento dos dados realizado com sucesso!")

# Consulta SQL para gerar o JSON estruturado
query_json = """
SELECT
    e.Empresa AS empresa,
    SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) = '1' THEN mc.Valor ELSE 0 END) AS total_ativo,
    SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) != '1' THEN mc.Valor ELSE 0 END) AS total_passivo,
    (SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) = '1' THEN mc.Valor ELSE 0 END) +
     SUM(CASE WHEN substr(mc."Conta Contábil", 1, 1) != '1' THEN mc.Valor ELSE 0 END)) AS check_integridade
FROM MovimentoContabil mc
JOIN Empresa e ON mc.Empresa = e.Codigo
GROUP BY e.Empresa;
"""

df_json = pd.read_sql_query(query_json, conn)

# Converter para JSON
result_json = df_json.to_json(orient='records', indent=4)

# Salvar JSON
json_path = '/content/resultado_balancete.json'
with open(json_path, 'w') as f:
    f.write(result_json)

print(f"Arquivo JSON gerado com sucesso! Caminho: {json_path}")

# Abrir o JSON em uma visualização HTML interativa
with open(json_path, 'r') as f:
    json_content = json.load(f)
    html_content = json2html.convert(json = json_content)
    display.display(display.HTML(html_content))

# Fechar a conexão com o banco de dados
conn.close()


Tabelas criadas com sucesso!
Dados inseridos com sucesso!
Tratamento dos dados realizado com sucesso!
Arquivo JSON gerado com sucesso! Caminho: /content/resultado_balancete.json


empresa,total_ativo,total_passivo,check_integridade
EMPRESA 1,2025109140.4,12237427.98,2037346568.38
EMPRESA 2,184290832.0399999,56767969.66000001,241058801.69999993
EMPRESA 3,-7051.2,-9924192.03,-9931243.229999999
EMPRESA 4,1497706894.3999999,1160661.7199999997,1498867556.12
EMPRESA 5,1876694707.36,-323909775.46,1552784931.8999999
